### A few introductory words

As per usual I start exploring my data with a fresh look that is not seeing already done EDAs of other users. I am sure kagglers have done amzing analysis and I will make sure to include those when I look into those. Hopefully at least some parts of my work will be usefull :)

In [ ]:
import os
import numpy as np
import pandas as pd
import dask.array as da
import dask.dataframe as dd

from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.plotting.backend = "plotly"

In [ ]:
seed = 987

In [ ]:
train_df = dd.read_csv('../input/jane-street-market-prediction/train.csv')
features_df = pd.read_csv('../input/jane-street-market-prediction/features.csv')
test_df = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')

In [ ]:
train_df.head(3)
features_df.head(3)

In [ ]:
train_df.info() 
features_df.info()

In [ ]:
features_df.shape
features_df.head()

In [ ]:
1*features_df.drop('feature', axis=1).values.sum(axis=1)
1*features_df.drop('feature', axis=1).values.sum(axis=0)

In [ ]:
sample = train_df.sample(frac=0.05, random_state=seed).compute()
fig = go.Figure(data=go.Heatmap(
                    z=sample.corr(),
                    x=sample.corr().columns,
                    y=sample.corr().columns))
fig.show()

In [ ]:
sample.weight[sample.weight>0].min()
sample.weight.max()

In [ ]:
sample.describe()

In [ ]:
ft_groups = sample.isna().sum()[sample.isna().sum()!=0]
ft_groups.head(20)

That is something interesting. Seems like some feautres are grouped, notice for exemple `feature_17` and `feature_18` have the same number of missing values. Coincidence? Seems unlikely, so I decided to store them as groups for later use.

In [ ]:
sample[['feature_17', 'feature_18']].head(10)

In [ ]:
features_df[(features_df.feature=='feature_17')|(features_df.feature=='feature_18')].T

In [ ]:
ft_group_cols = dict()
for val in ft_groups.unique():
    cols = ft_groups[ft_groups==val].index
    
    if len(cols) > 1:
        if val in ft_group_cols.keys():
            ft_group_cols[val] = ft_group_cols[val].add(cols)
        else:
            ft_group_cols[val] = set(cols)
ft_group_cols

In [ ]:
group_col = ft_group_cols[3525]
group_col.add('date')
group = sample[group_col].sort_values('date')
group = sample[[c for c in ft_group_cols[3525]]].set_index('date')
plt.plot(group, alpha=0.6);

Work in progress...